### Import all required libraries and set constants for server connection

In [ ]:
USE_LOCAL_SERVER = True
import os, sys
import numpy as np

import matplotlib.pyplot as plt

import time
while os.getcwd()[-6:] != 'inkube':
    os.chdir('..')
print(os.getcwd())
sys.path.insert(0, './Communication')
from Communication_for_stimulation import Communication
# Once the server is running:
if USE_LOCAL_SERVER:
    host     = '127.0.0.1' 
else:
    host     = '192.168.X.X'     
port = 0x1240

### Start an instance of the communication class which does all communication to the networks for you

In [ ]:
# Building the communication:
com = Communication(host,port)

## Spontaneous recording

In [ ]:
max_iterations = 2
counter = 0
all_responses = []

com.send_control({"mode": 1})

while counter < max_iterations:
    r = com.get_spont() # chunks are roughly 1.5 s max
    all_responses.append(r)
    print("Received Spikes")
    counter += 1

com.send_control({"mode": 0})

## Open loop stimulation pulse and parameters

In [ ]:
# create stim matrix of form [[number of samples before recording start, network_id (starting at 0), electrode_id (starting at 0)]]

stimulus = np.zeros((15,3), dtype=int)
stimulus[:,0] = 29
stimulus[:,1] = np.arange(15)
stimulus[:,2] = 0

print(stimulus[-1])

In [ ]:
com.send_control({"mode": 0, 'stim_amp': 1})

Closed loop stimulation cycle

In [ ]:
all_responses = []

time.sleep(2)

current_index = 0
iterations_max = 200

com.send_control({"mode": 2})

counter = 0
while counter < iterations_max:
    r = com.get_response()
    current_index = r["index"]
    com.send_stimulus(stimulus, current_index+1)
    if r["stim_recv"]:
        counter+=1
        all_responses.append(r)

time.sleep(1)
com.send_control({"mode": 0})

rasterplot of stimulus response of first MEA

In [ ]:
cols = ['red', 'green', 'blue', 'purple']

for nw in range(15):
    plt.figure()
    for id, a in enumerate(all_responses[0:]):
        for el in range(4):
            s = a['spikes'][nw,el]
            plt.scatter(s, id*np.ones(len(s)),c=cols[el],s=4)
    plt.xlim(0, 25)
    plt.title(f'{nw}')
plt.show()

## Open Loop stimulation with digital aux signal

In [ ]:
fs = 17_361 # sampling frequency
stim_cycle = .25 # in sec
MAX_PKG_ID = 1_562_500 # after this package IDs repeat. Important for dig aux

def get_pkg_from_index(index): # translate the stimulation cycle into the package ID as digital aux is controlled with package ID
    samples_per_cycle = stim_cycle*17_361
    pkg = index*samples_per_cycle
    return int(pkg)

alignment_mock_stim = np.array([[0,1,0]]) # use an unused MEA slot if available for this

num_of_electrodes = 4 # number of electrodes per circuit for pattern
max_num_of_pulses = 12 # maximum number of pulses to be sent open-loop

In [ ]:
delay = 10e-3 # in s, time between pulses
stim_delays = (np.arange(max_num_of_pulses)*delay*fs).astype(int) # delays of pulses in packages

digaux_time = 50e-3 # time before first pulse that digaux goes high, make sure this does not cross the stimulation cycle
digaux_duration =  10e-3 # duration of trigger pulse
digaux_delay_on = int(stim_delays[-1]+digaux_time*fs)
digaux_delay_off = int(stim_delays[-1]+(digaux_time-digaux_duration)*fs)


stim_times = stim_delays/fs
assert np.max(stim_times) < stim_cycle # stim_delays should not exceed the stim_cycle

create stimulation pulses with different timepoints

In [ ]:
# create stim matrix of form [[package_ID_delay, network_id (starting at 0), electrode_id (starting at 0)]]
stimulus_template = np.zeros((max_num_of_pulses,3), dtype=int)
all_stimuli = {}

stimulus_template[:,0] = stim_delays # set delays
# newtork stays at 0

# all at the same time
stimulus_all = np.repeat(stimulus_template, repeats=num_of_electrodes*np.ones(max_num_of_pulses, dtype=int), axis=0)
stimulus_all[:,2] = np.arange(num_of_electrodes*max_num_of_pulses, dtype=int)%num_of_electrodes # all electrodes at any timeslot
all_stimuli['all'] = stimulus_all

# cycle iterates through the electrodes in a cycle one after the other
stimulus_cycle = np.copy(stimulus_template)
stimulus_cycle[:,2] = np.arange(max_num_of_pulses, dtype=int)%num_of_electrodes
all_stimuli['cycle'] = stimulus_cycle

# sequence is same as all but for one electrode only
for seq_el in range(num_of_electrodes):
    stimulus_seq = np.copy(stimulus_template)
    stimulus_seq[:,2] = seq_el*np.ones(max_num_of_pulses, dtype=int)
    all_stimuli[f'seq_{seq_el}'] = stimulus_seq

In [ ]:
# adjust how many pulse trains are sent
iterations_max = 20
# adjust how many pulses one train contains
number_of_pulses = 12 # must be smaller than maximum 
# adjust what type of train is sent
stimulus = all_stimuli['all'] # select here 'all', 'cycle' or 'seq_x'
stimulus = stimulus[-(stimulus.shape[0]//max_num_of_pulses*number_of_pulses):]

Open loop stimulation cycle

In [ ]:
counter = 0
iterations = 0
current_index = 0
all_responses = []

# switch to open-loop stimulation mode
com.send_control({'mode': 3})

# align to match the right period, therefore a dummy pulse is sent
flag = 1
r = com.get_response()
while flag:
    com.send_stimulus(alignment_mock_stim, r["index"]+1)
    r = com.get_response()
    print('Sent alignment')
    flag = not r["stim_recv"]
#r = com.get_response()
while counter < iterations_max:
# for iterations in range(iterations_max):
    
    current_index = r["index"]
    if current_index > -1: # skip the first period
        com.send_control(
            {
                'digaux_on': (get_pkg_from_index(current_index+1)-digaux_delay_on)%MAX_PKG_ID,
                'digaux_off': (get_pkg_from_index(current_index+1)-digaux_delay_off)%MAX_PKG_ID 
            }
        )
        time.sleep(10e-3) # make sure the dig aux commands arrive and get executed before the stimulus
        com.send_stimulus(stimulus, current_index+1)
        print(f"Index for stim {current_index+1}")
    
    if r["stim_recv"]:
        counter+=1
        all_responses.append(r)
    iterations += 1
    print(f"Received {counter} / {iterations}. Index {current_index}")
    r = com.get_response()

time.sleep(2)
com.send_control({'mode': 0})